In [6]:
# ---------- To run the program just click 'Run All' and the file opener will come up (make sure to check your background applications) 
# ---------- Select the image containing the Shark and the enhanced shark will show up (once more make sure to check your background applications)
# ---------- To stop the image, press any key or close it manually 
import cv2
import easygui
from matplotlib import pyplot as plt
import numpy as np

# Read an image
I = easygui.fileopenbox(filetypes=["*.jpg","*.jpeg","*.png"])
I = cv2.imread(I)

# Resize image in case is too large or small
down_width = 1000
down_height = 500
down_points = (down_width, down_height)
I = cv2.resize(I, down_points, interpolation= cv2.INTER_LINEAR)

# Convert image to gray 
grayImage = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

# Sharpen image using a kernel 
k = np.array([[0,-1,0], 
              [-1,5,-1], 
              [0,-1,0]])
sharpened = cv2.filter2D(grayImage, ddepth=-1, kernel=k)

# equalize image histogram
grayImage = cv2.equalizeHist(sharpened)

# Gaussian blur on the image
grayImage = cv2.GaussianBlur(grayImage,(5,5),0)

In [7]:
hsv = cv2.cvtColor(I, cv2.COLOR_BGR2HSV)
  
# Creating mask1 by defining the lower and upper values of HSV, this will detect blue colour
lower_blue = np.array([100,251,170])
upper_blue = np.array([110,255,255])
Mask1 = cv2.inRange(hsv, lower_blue, upper_blue)

# Creating Mask2 with simple threshold 
T = np.mean(I) + np.std(I)
T, mask = cv2.threshold(I, thresh = T+8, maxval = 255, type = cv2.THRESH_BINARY)

# Convert Mask2 to grayscale 
gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

Mask2 = cv2.bitwise_and(grayImage, grayImage, mask = gray_mask)

# Creating Mask3 with adaptative threshold 
Mask3 = cv2.adaptiveThreshold(grayImage, maxValue = 255,adaptiveMethod = cv2.ADAPTIVE_THRESH_GAUSSIAN_C,thresholdType = cv2.THRESH_BINARY,blockSize = 205, C = 8)

# Combining Mask1, Mask2 and Mask3 
ROI = cv2.bitwise_or(Mask1, Mask2, Mask3)

# Finding contours 
contours, hierarchy = cv2.findContours(ROI, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

area_array = []
count = 1
# Make list with contourArea to check the area of the countour  
for i, j in enumerate(contours):
    area = cv2.contourArea(j)
    area_array.append(area)

# sorting array from biggest to smallest by area using lambda function to order by first element in list (Area)
sorted_array = sorted(zip(area_array, contours), key=lambda x: x[0], reverse=True)

# Using the second largest countour, most likely to be the shark
should_be_shark = sorted_array[1][1]

# Draw all contours (mostly for test)
image = cv2.drawContours(ROI, should_be_shark, -1, (0, 0, 0), 1)

# get bounding box, used to draw an approximate box around the ROI of the image
x,y,w,h = cv2.boundingRect(should_be_shark)

# crop the image at the bounds
crop = grayImage[y:y+h, x:x+w]

# Scale Shark
height, width = crop.shape[:2]
crop = cv2.resize(crop, dsize=(width * 2, height * 2))

# Displaying the shark
cv2.imshow('''RUN It's a Shark!!''', crop)
cv2.waitKey(0)  
cv2.destroyAllWindows()